In [20]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy

In [21]:
# read in data
df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")
df = df[df["volume"] != 0]
df.reset_index(drop = True, inplace = True)
df.isna().sum
df.head()

,Local time,open,high,low,close,volume
0,05.05.2003 00:00:00.000 GMT+0300,1.12354,1.13019,1.12030,1.12804,1053162.797
1,06.05.2003 00:00:00.000 GMT+0300,1.12798,1.14510,1.12738,1.14368,1061122.699
2,07.05.2003 00:00:00.000 GMT+0300,1.14367,1.14448,1.13275,1.13677,1056796.590
3,08.05.2003 00:00:00.000 GMT+0300,1.13675,1.15092,1.13026,1.15026,1058908.699
4,09.05.2003 00:00:00.000 GMT+0300,1.15002,1.15376,1.14362,1.14907,1063989.000


In [22]:
df.tail()

,Local time,open,high,low,close,volume
4729,24.06.2021 00:00:00.000 GMT+0300,1.19267,1.19565,1.19178,1.19322,85152.21000
4730,25.06.2021 00:00:00.000 GMT+0300,1.19322,1.19754,1.19264,1.19392,77837.64500
4731,28.06.2021 00:00:00.000 GMT+0300,1.19380,1.19447,1.19025,1.19260,85154.26000
4732,29.06.2021 00:00:00.000 GMT+0300,1.19297,1.19334,1.18779,1.18973,98898.57000
4733,30.06.2021 00:00:00.000 GMT+0300,1.18973,1.19092,1.18452,1.18589,4301.30191


In [23]:
df["signal"] = np.random.random(len(df))
df["signal"] = df["signal"].apply(lambda x : 1 if x < 0.5 else 2)

In [24]:
df[df["signal"] == 1].count() + df[df["signal"] == 2].count()

Local time    4734
open          4734
high          4734
low           4734
close         4734
volume        4734
signal        4734
dtype: int64

In [25]:
def SIGNAL():
    return df.signal

df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "signal"]

class my_strat(Strategy):
    initsize = 1
    mysize = initsize
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()

        if(self.signal1 > 0 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            self.mysize *= 2
        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            self.mysize = self.initsize
        
        if self.signal1 == 2 and len(self.trades) == 0:
            sl1 = self.data.Close[-1] - 600e-4
            tp1 = self.data.Close[-1] + 600e-4
            self.buy(sl = sl1, tp = tp1, size = self.mysize)

        if self.signal1 == 1 and len(self.trades) == 0:
            sl1 = self.data.Close[-1] - 600e-4
            tp1 = self.data.Close[-1] + 600e-4
            self.buy(sl = sl1, tp = tp1, size = self.mysize)

In [26]:
from backtesting import Backtest
bt = Backtest(df, my_strat, cash = 10000, commission = .00)
stat = bt.run()
stat

C:\Users\CKitc\AppData\Local\Temp\ipykernel_4304\4120176928.py:2: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, my_strat, cash = 10000, commission = .00)


TypeError: Can't instantiate abstract class my_strat with abstract method init